In [28]:
from tokenize import tokenize

import pandas as pd
from transformers import AutoTokenizer,AutoModelForSequenceClassification,Trainer,TrainingArguments
from sklearn.model_selection import train_test_split
import torch
from datasets import Dataset
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [25]:
df = pd.read_csv("cleaned_data.csv")
train_texts, test_texts, train_labels, test_labels = train_test_split(df["review"].tolist(),df["label"].tolist(), test_size=0.2)

In [31]:
tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-mini")
def preprocess_data(texts,labels):
    encodings = tokenizer(texts,padding="max_length",truncation=True,max_length=128)
    encodings["labels"] = labels
    return Dataset.from_dict(encodings)
train_dataset = preprocess_data(train_texts,train_labels)
test_dataset = preprocess_data(test_texts,test_labels)

model = AutoModelForSequenceClassification.from_pretrained("prajjwal1/bert-mini",num_labels=2)

config.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

C:\Users\saketh\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\saketh\.cache\huggingface\hub\models--prajjwal1--bert-mini. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/45.1M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-mini and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [39]:
training_args = TrainingArguments(output_dir="./output",num_train_epochs=1,per_device_train_batch_size=32,
                                  per_device_eval_batch_size=32,
                                  eval_strategy="epoch",
                                  logging_dir="./logs",
                                  logging_steps=100,
                                  load_best_model_at_end=True,
                                  save_total_limit=1,
                                  save_strategy="epoch",)
trainer = Trainer(model=model,args=training_args,train_dataset=train_dataset,eval_dataset=test_dataset,)
trainer.train()

model.save_pretrained("./sentiment_analysis")
tokenizer.save_pretrained("./sentiment_analysis")

Epoch,Training Loss,Validation Loss
1,0.358600,0.345289


('./sentiment_analysis\\tokenizer_config.json',
 './sentiment_analysis\\special_tokens_map.json',
 './sentiment_analysis\\vocab.txt',
 './sentiment_analysis\\added_tokens.json',
 './sentiment_analysis\\tokenizer.json')